In [1]:
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/master/Chapter03/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  4411k      0  0:00:02  0:00:02 --:--:-- 4413k


In [2]:
!pip uninstall -y tensorflow
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-og0zaepa
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-og0zaepa
  Resolved https://github.com/huggingface/transformers to commit fa6107c97edf7cf725305a34735a57875b67d85e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7638610 sha256=28cdf44b9f84247a86ce25b4ed42faec94bb6261607f6e94ded3e29099d80c85
  Stored in di

In [3]:
%%time
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer


paths = [str(x) for x in Path(".").glob("**/*.txt")]

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2,
special_tokens=[
  "<s>",
  "<pad>",
  "</s>",
  "<unk>",
  "<mask>",
])

CPU times: user 9.1 s, sys: 228 ms, total: 9.33 s
Wall time: 9.65 s


In [4]:
import os

token_dir = '/content/KantaiBERT'
if not os.path.exists(token_dir):
  os.makedirs(token_dir)
tokenizer.save_model('KantaiBERT')

['KantaiBERT/vocab.json', 'KantaiBERT/merges.txt']

In [5]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./KantaiBERT/vocab.json",
    "./KantaiBERT/merges.txt"
)
tokenizer.encode("The Thing is there.").tokens

['The', 'ĠTh', 'ing', 'Ġis', 'Ġthere', '.']

In [6]:
tokenizer.encode("The Thing is there.")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [7]:
tokenizer._tokenizer.post_processor = BertProcessing(
("</s>", tokenizer.token_to_id("</s>")),
("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [8]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [9]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['<s>', 'The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.', '</s>']

In [10]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [11]:
import torch
torch.cuda.is_available()

False

In [12]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)


In [13]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("./KantaiBERT",max_length=512)

In [14]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [15]:
params = list(model.parameters())
print(len(params))

106


In [16]:
for i in range(0,len(params)):
  print(params[i])

Parameter containing:
tensor([[-0.0129,  0.0080, -0.0394,  ...,  0.0101,  0.0152, -0.0181],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0275,  0.0098, -0.0029,  ...,  0.0039,  0.0164,  0.0059],
        ...,
        [ 0.0051,  0.0001, -0.0072,  ...,  0.0068, -0.0147,  0.0224],
        [-0.0163, -0.0044,  0.0178,  ...,  0.0235,  0.0098,  0.0057],
        [ 0.0151, -0.0096,  0.0052,  ...,  0.0291, -0.0037, -0.0008]],
       requires_grad=True)
Parameter containing:
tensor([[-1.6458e-02,  4.1858e-02, -6.4019e-03,  ..., -1.4583e-02,
          2.2110e-03,  3.8021e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 3.7084e-03,  1.7428e-02, -1.6278e-02,  ..., -9.4182e-03,
          1.5344e-02,  5.4015e-03],
        ...,
        [ 2.1930e-02,  5.8941e-03, -1.0445e-02,  ...,  2.1911e-02,
          1.8450e-02,  3.0282e-02],
        [ 1.2615e-02,  1.0481e-02, -2.6053e-02,  ...,  3.3144e-02,
   

In [17]:
n = 0
for i in range(0,len(params)):
  is_list = True
  try:
    list2 =len(params[i][0])
  except:
    list2 =1
    is_list=False
  list1=len(params[i])
  list3=list1*list2
  n += list3
  if is_list == True:
    print(i,list1,list2,list3)
  else:
    print(i,list1,list3)
print(n)

0 52000 768 39936000
1 514 768 394752
2 1 768 768
3 768 768
4 768 768
5 768 768 589824
6 768 768
7 768 768 589824
8 768 768
9 768 768 589824
10 768 768
11 768 768 589824
12 768 768
13 768 768
14 768 768
15 3072 768 2359296
16 3072 3072
17 768 3072 2359296
18 768 768
19 768 768
20 768 768
21 768 768 589824
22 768 768
23 768 768 589824
24 768 768
25 768 768 589824
26 768 768
27 768 768 589824
28 768 768
29 768 768
30 768 768
31 3072 768 2359296
32 3072 3072
33 768 3072 2359296
34 768 768
35 768 768
36 768 768
37 768 768 589824
38 768 768
39 768 768 589824
40 768 768
41 768 768 589824
42 768 768
43 768 768 589824
44 768 768
45 768 768
46 768 768
47 3072 768 2359296
48 3072 3072
49 768 3072 2359296
50 768 768
51 768 768
52 768 768
53 768 768 589824
54 768 768
55 768 768 589824
56 768 768
57 768 768 589824
58 768 768
59 768 768 589824
60 768 768
61 768 768
62 768 768
63 3072 768 2359296
64 3072 3072
65 768 3072 2359296
66 768 768
67 768 768
68 768 768
69 768 768 589824
70 768 768
71 768 768

In [18]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='./kant.txt',
    block_size=128
)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 32.1 s, sys: 464 ms, total: 32.6 s
Wall time: 34.2 s


In [19]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm = True, mlm_probability=0.15
)

In [25]:
! pip install -U accelerate
! pip install -U transformers

In [27]:
import accelerate
accelerate.__version__

'0.22.0'

In [28]:
from transformers import Trainer, TrainingArguments

train_arg = TrainingArguments(
    output_dir = "./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args= train_arg,
    data_collator=data_collator,
    train_dataset=dataset
)

ImportError: ignored

In [ ]:
%%time
trainer.train()

In [ ]:
trainer.save_model("./KantaiBERT")